In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import re
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime ,timedelta

# Exploratory Data Analysis of the merdged data

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Quantium/final.csv', index_col=0)

In [ ]:
data.shape

(264834, 13)

In [ ]:
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,customer_segment,family_type,age_group
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,Premium,Single/Couple,young
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,Mainstream,Single/Couple,young
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,Budget,Families,young
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,Budget,Families,young
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,Mainstream,Single/Couple,old


The trail stores are **77, 86, 88**. got to find control stores accordingly

In [ ]:
data['DATE'] = data['DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
data['MONTH'] = data['DATE'].apply(lambda x: x.month )
data['YEAR'] = data['DATE'].apply(lambda x: x.year )

Calculating the metrics that can be used to compare the stores

In [ ]:
# Foot Traffic (total number of walk-ins)
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(no_of_transaction=('TXN_ID',pd.Series.nunique))

In [ ]:
# Unique customers
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(unique_customers=('LYLTY_CARD_NBR',pd.Series.nunique))

In [ ]:
# total units sold
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(total_units = ('PROD_QTY',np.sum))

In [ ]:
# total sales 
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(total_sales = ('TOT_SALES',np.sum))

In [ ]:
# Popular brand
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(popular_brand = ('BRAND',pd.Series.mode))

In [ ]:
# Popular brand
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(popular_segment = ('customer_segment',pd.Series.mode))

In [ ]:
# Popular brand
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(popular_family_type = ('family_type',pd.Series.mode))

In [ ]:
# Popular brand
data.groupby(['YEAR','MONTH','STORE_NBR']).agg(popular_age = ('age_group',pd.Series.mode))

In [ ]:
# combining all the above so that i can get a dataframe
metric = data.groupby(['YEAR','MONTH','STORE_NBR']).agg(no_of_transaction=('TXN_ID',pd.Series.nunique), unique_customers=('LYLTY_CARD_NBR',pd.Series.nunique),
                                               total_units = ('PROD_QTY',np.sum), total_sales = ('TOT_SALES',np.sum),
                                               popular_brand = ('BRAND',pd.Series.mode), popular_segment = ('customer_segment',pd.Series.mode),
                                               popular_family_type = ('family_type',pd.Series.mode), popular_age = ('age_group',pd.Series.mode))

In [ ]:
metric.head()
# popular brand, etc can't be used for selecting the control store as i think it has some calculation problem ?

no_of_transaction  ...  popular_age
YEAR MONTH STORE_NBR                     ...             
2018 7     1                         52  ...        young
           2                         41  ...        young
           3                        138  ...          old
           4                        158  ...          old
           5                        120  ...          old

[5 rows x 8 columns]

In [ ]:
# Calculating units per transaction
metric['units_per_txn']=metric['total_units']/metric['no_of_transaction']

In [ ]:
# Calculating price per unit
metric['price_per_unit'] = metric['total_sales']/metric['total_units']

In [ ]:
# Calculating transaction per customer
metric['Txn_per_customer']=metric['no_of_transaction']/ metric['unique_customers']

In [ ]:
part1 = metric.loc[2018,:,:]

In [ ]:
part2 = metric.loc[2019,:1,:]

In [ ]:
# performance from the non trial period. use this to calculate the control store
non_trial = pd.concat([part1,part2])

In [ ]:
# only considering the numerical metric
non_trial.drop(['popular_brand','popular_segment','popular_family_type','popular_age'], axis=1, inplace=True)

In [ ]:
non_trial.reset_index(inplace=True)

In [ ]:
non_trial.head() 

,YEAR,MONTH,STORE_NBR,no_of_transaction,unique_customers,total_units,total_sales,units_per_txn,price_per_unit,Txn_per_customer
0,2018,7,1,52,49,62,206.9,1.192308,3.337097,1.061224
1,2018,7,2,41,39,46,150.8,1.121951,3.278261,1.051282
2,2018,7,3,138,112,271,1205.7,1.963768,4.449077,1.232143
3,2018,7,4,158,128,318,1399.9,2.012658,4.402201,1.234375
4,2018,7,5,120,93,240,812.0,2.000000,3.383333,1.290323


In [ ]:
non_trial['STORE_NBR']=non_trial['STORE_NBR'].apply(lambda x: int(x))

In [ ]:
non_trial.to_csv('/content/drive/MyDrive/Quantium/metric.csv')

In [ ]:
# using the given method - CORRELATION

def calculate_correlation(inputTable, metric, trial_store):
  output = pd.DataFrame({'Store1':[], 'Store2':[], 'Correlation':[]})
  a = inputTable.loc[inputTable['STORE_NBR'] == trial_store, metric]
  a.reset_index(drop=True, inplace=True)
  storeNumbers = inputTable['STORE_NBR'].unique()
  for i in storeNumbers:
    b = inputTable.loc[inputTable['STORE_NBR'] == i, metric]
    b.reset_index(drop=True, inplace=True)
    output = output.append({'Store1':trial_store, 'Store2':i, 'Correlation':b.corr(a)}, ignore_index = True)
  return output

In [ ]:
# using the given method - DISTANCE

def calculate_magnitude(inputTable, metric, trial_store):
  output = pd.DataFrame({'Store1':[], 'Store2':[], 'Magnitude':[]})
  a = inputTable.loc[inputTable['STORE_NBR'] == trial_store, metric]
  a.reset_index(drop=True, inplace=True)
  storeNumbers = inputTable['STORE_NBR'].unique()
  for i in storeNumbers:
    b = inputTable.loc[inputTable['STORE_NBR'] == i, metric]
    b.reset_index(drop=True, inplace=True)
    c = abs(a-b)
    # scaling the value of c, 1 - as we want to sort in descending ?
    d = np.mean(1-(c - min(c))/(max(c) - min(c)))
    output = output.append({'Store1':trial_store, 'Store2':i, 'Magnitude':d}, ignore_index = True)
  return output

# for Trail Store 77

## Correlation

In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 77).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
73,77.0,77.0,1.000000
227,77.0,233.0,0.958422
114,77.0,119.0,0.940070
2,77.0,3.0,0.903701
152,77.0,157.0,0.895690
68,77.0,71.0,0.867547
247,77.0,254.0,0.865382
38,77.0,41.0,0.843928
15,77.0,17.0,0.808263
110,77.0,115.0,0.768323


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'unique_customers', trial_store = 77).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
73,77.0,77.0,1.000000
227,77.0,233.0,0.990358
114,77.0,119.0,0.983267
247,77.0,254.0,0.916208
108,77.0,113.0,0.901348
80,77.0,84.0,0.858571
38,77.0,41.0,0.844219
2,77.0,3.0,0.834207
32,77.0,35.0,0.774647
84,77.0,88.0,0.765048


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'total_units', trial_store = 77).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
73,77.0,77.0,1.000000
114,77.0,119.0,0.975135
227,77.0,233.0,0.820381
25,77.0,27.0,0.814323
116,77.0,121.0,0.802161
68,77.0,71.0,0.796521
36,77.0,39.0,0.795772
2,77.0,3.0,0.744618
38,77.0,41.0,0.737611
250,77.0,257.0,0.731562


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'total_sales', trial_store = 77).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
73,77.0,77.0,1.000000
68,77.0,71.0,0.914106
227,77.0,233.0,0.903774
114,77.0,119.0,0.867664
15,77.0,17.0,0.842668
2,77.0,3.0,0.806644
38,77.0,41.0,0.783232
47,77.0,50.0,0.763866
152,77.0,157.0,0.735893
157,77.0,162.0,0.729740


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'units_per_txn', trial_store = 77).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)
# not gonna use this as the correlation is < 0.9 and as store 233 and 199 are the ones with the most correlation in above and they are not here.

,Store1,Store2,Correlation
73,77.0,77.0,1.000000
6,77.0,7.0,0.832579
118,77.0,123.0,0.791055
149,77.0,154.0,0.697123
140,77.0,145.0,0.658477
141,77.0,146.0,0.621986
138,77.0,143.0,0.604845
18,77.0,20.0,0.585255
15,77.0,17.0,0.584553
257,77.0,264.0,0.578541


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'price_per_unit', trial_store = 77).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)
# not gonna use this as the correlation is < 0.9 and as store 233 and 199 are the ones with the most correlation in above.

,Store1,Store2,Correlation
73,77.0,77.0,1.000000
152,77.0,157.0,0.894507
155,77.0,160.0,0.787038
255,77.0,262.0,0.750477
21,77.0,23.0,0.743961
239,77.0,245.0,0.706632
192,77.0,198.0,0.698082
251,77.0,258.0,0.697247
139,77.0,144.0,0.650623
201,77.0,207.0,0.625411


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'Txn_per_customer', trial_store = 77).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)
# not gonna use this as the correlation is < 0.9 and as store 233 and 199 are the ones with the most correlation in above.

,Store1,Store2,Correlation
73,77.0,77.0,1.000000
171,77.0,176.0,0.881098
139,77.0,144.0,0.868818
82,77.0,86.0,0.850527
113,77.0,118.0,0.817083
176,77.0,181.0,0.813600
114,77.0,119.0,0.802529
207,77.0,213.0,0.763779
11,77.0,13.0,0.758140
185,77.0,190.0,0.751048


## Distance

In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
61,77.0,64.0,0.752381
70,77.0,73.0,0.751880
259,77.0,266.0,0.728571
131,77.0,136.0,0.727273
48,77.0,51.0,0.726708
241,77.0,247.0,0.725869
66,77.0,69.0,0.723214
156,77.0,161.0,0.723214
0,77.0,1.0,0.722689
119,77.0,124.0,0.714286


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'unique_customers', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
253,77.0,260.0,0.757764
61,77.0,64.0,0.740260
45,77.0,48.0,0.730159
48,77.0,51.0,0.727891
166,77.0,171.0,0.714286
53,77.0,56.0,0.709360
33,77.0,36.0,0.708791
17,77.0,19.0,0.707641
258,77.0,265.0,0.705882
119,77.0,124.0,0.700000


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'total_units', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
16,77.0,18.0,0.782468
214,77.0,220.0,0.776786
48,77.0,51.0,0.767347
208,77.0,214.0,0.758242
54,77.0,57.0,0.743386
57,77.0,60.0,0.735065
239,77.0,245.0,0.726444
140,77.0,145.0,0.721805
66,77.0,69.0,0.720588
25,77.0,27.0,0.714286


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'total_sales', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
16,77.0,18.0,0.733070
110,77.0,115.0,0.727536
66,77.0,69.0,0.720729
40,77.0,43.0,0.715421
113,77.0,118.0,0.712821
48,77.0,51.0,0.711352
208,77.0,214.0,0.698926
187,77.0,192.0,0.690417
241,77.0,247.0,0.684843
109,77.0,114.0,0.679845


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'units_per_txn', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)
# not gonna use this as the correlation is < 0.9 and as store 233 and 199 are the ones with the most correlation in above and they are not here.

,Store1,Store2,Magnitude
182,77.0,187.0,0.753252
98,77.0,103.0,0.744092
156,77.0,161.0,0.734413
5,77.0,6.0,0.722518
192,77.0,198.0,0.717830
141,77.0,146.0,0.698750
227,77.0,233.0,0.683447
17,77.0,19.0,0.675673
43,77.0,46.0,0.667777
41,77.0,44.0,0.666096


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'price_per_unit', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)
# not gonna use this as the correlation is < 0.9 and as store 233 and 199 are the ones with the most correlation in above.

,Store1,Store2,Magnitude
218,77.0,224.0,0.761399
18,77.0,20.0,0.719093
187,77.0,192.0,0.718739
43,77.0,46.0,0.715088
255,77.0,262.0,0.711045
12,77.0,14.0,0.699054
117,77.0,122.0,0.692844
259,77.0,266.0,0.689685
48,77.0,51.0,0.688511
158,77.0,163.0,0.687856


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'Txn_per_customer', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)
# not gonna use this as the correlation is < 0.9 and as store 233 and 199 are the ones with the most correlation in above.

,Store1,Store2,Magnitude
253,77.0,260.0,0.786707
247,77.0,254.0,0.754644
8,77.0,9.0,0.749375
194,77.0,200.0,0.748203
43,77.0,46.0,0.746348
222,77.0,228.0,0.737893
228,77.0,234.0,0.731308
262,77.0,269.0,0.724397
23,77.0,25.0,0.706615
227,77.0,233.0,0.699250


Distance is not a good measure i think as no same store is coming on top consistently. Not going to distance as not getting consistent store number, The control store for trial store 77 can be Store number 233, 119

# for Trail Store 86

## Correlation

In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 86).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
82,86.0,86.0,1.000000
133,86.0,138.0,0.819280
20,86.0,22.0,0.807787
142,86.0,147.0,0.799353
161,86.0,166.0,0.765179
171,86.0,176.0,0.742000
241,86.0,247.0,0.718834
214,86.0,220.0,0.708999
71,86.0,74.0,0.692247
46,86.0,49.0,0.690542


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'unique_customers', trial_store = 86).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
82,86.0,86.0,1.000000
150,86.0,155.0,0.942876
109,86.0,114.0,0.855339
253,86.0,260.0,0.846502
171,86.0,176.0,0.796380
104,86.0,109.0,0.770778
133,86.0,138.0,0.749701
219,86.0,225.0,0.733791
53,86.0,56.0,0.703549
214,86.0,220.0,0.662055


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'total_units', trial_store = 86).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
82,86.0,86.0,1.000000
133,86.0,138.0,0.894762
20,86.0,22.0,0.886171
142,86.0,147.0,0.856792
72,86.0,75.0,0.849881
241,86.0,247.0,0.811503
109,86.0,114.0,0.810371
248,86.0,255.0,0.790386
96,86.0,101.0,0.752259
150,86.0,155.0,0.751529


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'total_sales', trial_store = 86).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
82,86.0,86.0,1.000000
150,86.0,155.0,0.877882
127,86.0,132.0,0.846517
41,86.0,44.0,0.837692
234,86.0,240.0,0.825066
216,86.0,222.0,0.795075
104,86.0,109.0,0.788300
133,86.0,138.0,0.759864
192,86.0,198.0,0.748794
109,86.0,114.0,0.734415


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'units_per_txn', trial_store = 86).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
82,86.0,86.0,1.000000
89,86.0,94.0,0.980752
64,86.0,67.0,0.956624
155,86.0,160.0,0.945376
148,86.0,153.0,0.935623
105,86.0,110.0,0.927647
59,86.0,62.0,0.923630
72,86.0,75.0,0.920335
147,86.0,152.0,0.918681
65,86.0,68.0,0.910155


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'price_per_unit', trial_store = 86).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
82,86.0,86.0,1.000000
68,86.0,71.0,0.844669
15,86.0,17.0,0.844202
33,86.0,36.0,0.814318
209,86.0,215.0,0.814111
233,86.0,239.0,0.813920
185,86.0,190.0,0.792529
199,86.0,205.0,0.783723
255,86.0,262.0,0.770012
162,86.0,167.0,0.684289


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'Txn_per_customer', trial_store = 86).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
82,86.0,86.0,1.000000
11,86.0,13.0,0.906296
113,86.0,118.0,0.903326
20,86.0,22.0,0.868221
73,86.0,77.0,0.850527
151,86.0,156.0,0.732355
140,86.0,145.0,0.728346
96,86.0,101.0,0.719032
139,86.0,144.0,0.676598
262,86.0,269.0,0.675767


Not going to distance as not getting consistent store number, The control store for trial store 86 can be Store number 138, 155

# for Trail Store 88

## Correlation

In [ ]:
m = calculate_magnitude(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 88).set_index('Store2')
c = calculate_correlation(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 88).set_index('Store2')

pd.DataFrame(m['Magnitude'] *0.5 + c['Correlation'] *0.5).sort_values(0, ascending=False).head(10)

,0
Store2,
7.0,0.767066
123.0,0.682433
134.0,0.674856
14.0,0.616837
237.0,0.604472
102.0,0.589766
204.0,0.583203
6.0,0.579573
97.0,0.576477


In [ ]:
m = calculate_magnitude(inputTable= non_trial, metric = 'total_sales', trial_store = 88).set_index('Store2')
c = calculate_correlation(inputTable= non_trial, metric = 'total_sales', trial_store = 88).set_index('Store2')

pd.DataFrame(m['Magnitude'] *0.5 + c['Correlation'] *0.5).sort_values(0, ascending=False).head(10)

,0
Store2,
134.0,0.775084
11.0,0.750000
31.0,0.750000
204.0,0.697512
159.0,0.692344
1.0,0.681297
253.0,0.666914
91.0,0.657912
178.0,0.650803


In [ ]:
m = calculate_magnitude(inputTable= non_trial, metric = 'unique_customers', trial_store = 88).set_index('Store2')
c = calculate_correlation(inputTable= non_trial, metric = 'unique_customers', trial_store = 88).set_index('Store2')

pd.DataFrame(m['Magnitude'] *0.5 + c['Correlation'] *0.5).sort_values(0, ascending=False).head(10)

,0
Store2,
35.0,0.783130
237.0,0.777235
178.0,0.707828
265.0,0.693234
41.0,0.686222
14.0,0.685774
69.0,0.679325
77.0,0.655251
268.0,0.651462


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 88).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
84,88.0,88.0,1.000000
129,88.0,134.0,0.956855
6,88.0,7.0,0.946829
118,88.0,123.0,0.906737
12,88.0,14.0,0.836849
198,88.0,204.0,0.815758
231,88.0,237.0,0.723230
108,88.0,113.0,0.693233
87,88.0,91.0,0.616161
74,88.0,78.0,0.615884


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'unique_customers', trial_store = 88).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
84,88.0,88.0,1.000000
231,88.0,237.0,0.947326
12,88.0,14.0,0.942976
173,88.0,178.0,0.939466
32,88.0,35.0,0.899594
108,88.0,113.0,0.862632
258,88.0,265.0,0.841013
66,88.0,69.0,0.815792
114,88.0,119.0,0.775468
36,88.0,39.0,0.768465


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'total_units', trial_store = 88).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
84,88.0,88.0,1.000000
129,88.0,134.0,0.947328
154,88.0,159.0,0.802034
0,88.0,1.0,0.764948
150,88.0,155.0,0.758963
173,88.0,178.0,0.726331
234,88.0,240.0,0.714826
94,88.0,99.0,0.710923
74,88.0,78.0,0.694009
109,88.0,114.0,0.691996


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'total_sales', trial_store = 88).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
84,88.0,88.0,1.000000
268,88.0,11.0,1.000000
267,88.0,31.0,1.000000
154,88.0,159.0,0.903186
198,88.0,204.0,0.885774
129,88.0,134.0,0.864293
0,88.0,1.0,0.813636
246,88.0,253.0,0.811838
87,88.0,91.0,0.776688
58,88.0,61.0,0.748929


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'units_per_txn', trial_store = 88).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
84,88.0,88.0,1.000000
67,88.0,70.0,0.986016
188,88.0,194.0,0.984235
100,88.0,105.0,0.984234
42,88.0,45.0,0.978974
113,88.0,118.0,0.976820
139,88.0,144.0,0.975970
111,88.0,116.0,0.973758
176,88.0,181.0,0.971462
26,88.0,28.0,0.970694


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'price_per_unit', trial_store = 88).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
268,88.0,11.0,1.000000
84,88.0,88.0,1.000000
267,88.0,31.0,1.000000
66,88.0,69.0,0.939320
24,88.0,26.0,0.866336
91,88.0,96.0,0.823923
187,88.0,192.0,0.804507
103,88.0,108.0,0.785233
127,88.0,132.0,0.715830
249,88.0,256.0,0.710118


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_correlation(inputTable= non_trial, metric = 'Txn_per_customer', trial_store = 88).sort_values(by = ['Correlation'], axis=0, ascending=False).head(10)

,Store1,Store2,Correlation
84,88.0,88.0,1.000000
132,88.0,137.0,0.886607
199,88.0,205.0,0.814837
237,88.0,243.0,0.808608
265,88.0,272.0,0.805722
101,88.0,106.0,0.803873
22,88.0,24.0,0.767075
164,88.0,169.0,0.746504
17,88.0,19.0,0.735663
79,88.0,83.0,0.724206


## Distance

In [ ]:
t = calculate_magnitude(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 88)
t[t['Store2'] == 237]

,Store1,Store2,Magnitude
231,88.0,237.0,0.485714


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'no_of_transaction', trial_store = 88).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
215,88.0,221.0,0.745174
213,88.0,219.0,0.742857
261,88.0,268.0,0.714286
133,88.0,138.0,0.687708
235,88.0,241.0,0.677741
95,88.0,100.0,0.671429
252,88.0,259.0,0.669388
37,88.0,40.0,0.668067
207,88.0,213.0,0.660099
216,88.0,222.0,0.657143


In [ ]:
t = calculate_magnitude(inputTable= non_trial, metric = 'unique_customers', trial_store = 88)
t[t['Store2'] == 237]

,Store1,Store2,Magnitude
231,88.0,237.0,0.607143


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'unique_customers', trial_store = 88).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
161,88.0,166.0,0.763547
230,88.0,236.0,0.741935
123,88.0,128.0,0.705882
213,88.0,219.0,0.701863
166,88.0,171.0,0.692857
99,88.0,104.0,0.687500
160,88.0,165.0,0.685714
2,88.0,3.0,0.683673
37,88.0,40.0,0.678571
222,88.0,228.0,0.672269


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'total_units', trial_store = 88).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
215,88.0,221.0,0.765458
235,88.0,241.0,0.735714
213,88.0,219.0,0.735294
5,88.0,6.0,0.719388
95,88.0,100.0,0.704633
33,88.0,36.0,0.681319
37,88.0,40.0,0.675510
133,88.0,138.0,0.673190
125,88.0,130.0,0.664452
147,88.0,152.0,0.660714


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'total_sales', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
16,77.0,18.0,0.733070
110,77.0,115.0,0.727536
66,77.0,69.0,0.720729
40,77.0,43.0,0.715421
113,77.0,118.0,0.712821
48,77.0,51.0,0.711352
208,77.0,214.0,0.698926
187,77.0,192.0,0.690417
241,77.0,247.0,0.684843
109,77.0,114.0,0.679845


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'units_per_txn', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
182,77.0,187.0,0.753252
98,77.0,103.0,0.744092
156,77.0,161.0,0.734413
5,77.0,6.0,0.722518
192,77.0,198.0,0.717830
141,77.0,146.0,0.698750
227,77.0,233.0,0.683447
17,77.0,19.0,0.675673
43,77.0,46.0,0.667777
41,77.0,44.0,0.666096


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable = non_trial, metric = 'price_per_unit', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
218,77.0,224.0,0.761399
18,77.0,20.0,0.719093
187,77.0,192.0,0.718739
43,77.0,46.0,0.715088
255,77.0,262.0,0.711045
12,77.0,14.0,0.699054
117,77.0,122.0,0.692844
259,77.0,266.0,0.689685
48,77.0,51.0,0.688511
158,77.0,163.0,0.687856


In [ ]:
# chosing the top ten with similar NUMBER OF TRANSACTION
calculate_magnitude(inputTable= non_trial, metric = 'Txn_per_customer', trial_store = 77).sort_values(by = ['Magnitude'], axis=0, ascending=False).head(10)

,Store1,Store2,Magnitude
253,77.0,260.0,0.786707
247,77.0,254.0,0.754644
8,77.0,9.0,0.749375
194,77.0,200.0,0.748203
43,77.0,46.0,0.746348
222,77.0,228.0,0.737893
228,77.0,234.0,0.731308
262,77.0,269.0,0.724397
23,77.0,25.0,0.706615
227,77.0,233.0,0.699250


The control store for trial store 88 can be Store number 134, 237. tried combining the two measures

# Visualising if the stores are actual control store

# trying to come up with my function. 

In [ ]:
median_metric = non_trial.groupby(['STORE_NBR']).median()

In [ ]:
median_metric.loc[237]

In [ ]:
median_metric['no_of_transaction'] - 44

In [ ]:
def outliers(data):
  q1 = np.percentile(data.dropna(), 25)
  q2 = np.percentile(data.dropna(), 50)
  q3 = np.percentile(data.dropna(), 75)
  iqr = q3 - q1
  lower = q1 - (1.5 * iqr)
  upper = q3 + (1.5 * iqr)
  return {'lower_bound': lower, 'upper_bound':upper }

In [ ]:
print('min',min(metric.loc[:,:,77]['no_of_transaction']),
      'Q1', np.percentile(metric.loc[:,:,77]['no_of_transaction'], 25), 
      'Q2', np.percentile(metric.loc[:,:,77]['no_of_transaction'], 50), 
      'Q3', np.percentile(metric.loc[:,:,77]['no_of_transaction'], 75),
      'max',max(metric.loc[:,:,77]['no_of_transaction']))

min 38 Q1 43.5 Q2 46.5 Q3 49.75 max 56


In [ ]:
print('min',min(metric.loc[:,:,233]['no_of_transaction']),
      'Q1', np.percentile(metric.loc[:,:,233]['no_of_transaction'], 25), 
      'Q2', np.percentile(metric.loc[:,:,233]['no_of_transaction'], 50), 
      'Q3', np.percentile(metric.loc[:,:,233]['no_of_transaction'], 75),
      'max',max(metric.loc[:,:,233]['no_of_transaction']))

min 32 Q1 39.75 Q2 43.0 Q3 50.0 max 62


In [ ]:
metric.loc[:,:,233]

no_of_transaction  unique_customers  ...  popular_family_type  popular_age
YEAR MONTH                                       ...                                  
2018 7                     54                51  ...        Single/Couple          old
     8                     50                48  ...        Single/Couple        young
     9                     45                42  ...        Single/Couple          old
     10                    36                35  ...        Single/Couple          old
     11                    41                40  ...        Single/Couple        young
     12                    50                47  ...        Single/Couple          old
2019 1                     35                35  ...        Single/Couple          old
     2                     47                45  ...        Single/Couple          old
     3                     41                40  ...        Single/Couple          old
     4                     32                30  ...        Single/Couple          old
     5                     62                57  ...        Single/Couple          old
     6                     41                41  ...        Single/Couple          old

[12 rows x 8 columns]

In [ ]:
metric.loc[:,:,77]

no_of_transaction  unique_customers  ...  popular_family_type  popular_age
YEAR MONTH                                       ...                                  
2018 7                     55                51  ...        Single/Couple          old
     8                     48                47  ...        Single/Couple          old
     9                     44                42  ...        Single/Couple          old
     10                    38                37  ...        Single/Couple          old
     11                    44                41  ...        Single/Couple          old
     12                    48                46  ...        Single/Couple          old
2019 1                     39                35  ...        Single/Couple        young
     2                     45                45  ...        Single/Couple          old
     3                     55                50  ...        Single/Couple          old
     4                     48                47  ...        Single/Couple          old
     5                     56                55  ...        Single/Couple          old
     6                     42                41  ...        Single/Couple        young

[12 rows x 8 columns]